In [1]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain_community.llms import GPT4All
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

mistral_openorca = "mistral-7b-openorca.gguf2.Q4_0.gguf"
llm_path = "C://Users/pogawal/AppData/Local/nomic.ai/GPT4All/" + mistral_openorca

In [ ]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=llm_path, callbacks=callbacks, verbose=True)
llm_chain = LLMChain(prompt=prompt, llm=llm)


question = "What are the basic disciplines in mathematics?"

llm_chain.run(question)

In [ ]:
question = "Explain the basic thoughts of Der logische Aufbau der Welt by Rudolf Carnap?"

answer = llm_chain.run(question)

In [ ]:
from IPython.display import display, Markdown
display(Markdown(answer))

In [8]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma



loader = WebBaseLoader("https://github.com/SAP/styleguides/blob/main/clean-abap/CleanABAP.md")
data = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(data)

length_texts = len(texts)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


db = Chroma.from_documents(texts, embeddings, persist_directory="db")

llm_path = "C://Users/pogawal/AppData/Local/nomic.ai/GPT4All/gpt4all-falcon-newbpe-q4_0.gguf"

llm = GPT4All(
    model=llm_path,
    backend="gptj",
    verbose=False
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": length_texts}),
    return_source_documents=True,
    verbose=False,
)

In [ ]:
res = qa(f"""
    What are the golden rules for ABAP development?.
""")
print(res["result"])